In [1]:
from isaacgym import gymtorch, gymapi
from isaacgymenvs import train
from rl_games.torch_runner import Runner
from hydra import compose, initialize
from rl_games.common import env_configurations, vecenv
from isaacgymenvs.utils.rlgames_utils import RLGPUEnv, RLGPUTaskAlgoObserver, get_rlgames_env_creator
from rl_games.algos_torch import model_builder
from isaacgymenvs.learning import common_player
from isaacgymenvs.utils.reformat import omegaconf_to_dict

import isaacgymenvs
import yaml
import torch
import numpy as np
import time
import os

os.environ['DISPLAY'] = ":5"

Importing module 'gym_37' (/scr-ssd/ksrini/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_37.so)
Setting GYM_USD_PLUG_INFO_PATH to /scr-ssd/ksrini/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 1.8.1
Device count 1
/scr-ssd/ksrini/Downloads/isaacgym/python/isaacgym/_bindings/src/gymtorch
Using /afs/cs.stanford.edu/u/ksrini/.cache/torch_extensions as PyTorch extensions root...
Emitting ninja build file /afs/cs.stanford.edu/u/ksrini/.cache/torch_extensions/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module gymtorch...


2022-07-14 07:38:04,547 - INFO - logger - logger initialized


Error: FBX library failed to load - importing FBX data will not succeed. Message: No module named 'fbx'
FBX tools must be installed from https://help.autodesk.com/view/FBX/2020/ENU/?guid=FBX_Developer_Help_scripting_with_python_fbx_installing_python_fbx_html


## Initialize config and defining builder helper functions

In [2]:
with initialize(config_path="cfg", job_name="test_env"):
    cfg = compose(config_name="config", overrides=["task=AllegroHandGrasp", 
                                                   "task.env.observationType=full",
                                                   # "task.env.objectType=spray_bottle",
                                                   # "sim_device=cpu",
                                                   "experiment=scaled-ac",
                                                   "headless=true",
                                                   "test=true",
                                                   "task.env.useRelativeControl=true",
                                                   "checkpoint=./runs/scaled-ac/nn/scaled-ac.pth",
                                                   "num_envs=64"])


def create_env_thunk(**kwargs):
        envs = isaacgymenvs.make(cfg.seed, cfg.task_name, cfg.task.env.numEnvs, 
            cfg.sim_device, cfg.rl_device, cfg.graphics_device_id, cfg.headless,
            cfg.multi_gpu, cfg.capture_video, cfg.force_render, cfg,
            **kwargs,
        )
        if cfg.capture_video:
            envs.is_vector_env = True
            envs = gym.wrappers.RecordVideo(
                envs,
                f"videos/{run_name}",
                step_trigger=lambda step: step % cfg.capture_video_freq == 0,
                video_length=cfg.capture_video_len,
            ) 
        return envs

# register new AMP network builder and agent
def build_runner(algo_observer):
    runner = Runner(algo_observer)
    runner.player_factory.register_builder(
        'a2c_continuous', lambda **kwargs : common_player.CommonPlayer(**kwargs)
    )
    return runner
        
# register the rl-games adapter to use inside the runner
vecenv.register('RLGPU',
                lambda config_name, num_actors, **kwargs: RLGPUEnv(config_name, num_actors, **kwargs))
env_configurations.register('rlgpu', {
    'vecenv_type': 'RLGPU',
    'env_creator': create_env_thunk,
})

/scr1/.pyenv/versions/miniconda3-latest/envs/rlgpu/lib/python3.7/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/scr1/.pyenv/versions/miniconda3-latest/envs/rlgpu/lib/python3.7/site-packages/hydra/_internal/defaults_list.py:413: UserWarning: In config: Invalid overriding of hydra/job_logging:
Default list overrides requires 'override' keyword.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/defaults_list_override for more information.

  deprecation_warning(msg)


## Create runner and player (agent)

In [3]:
runner = build_runner(RLGPUTaskAlgoObserver())
rlg_config_dict = omegaconf_to_dict(cfg.train)
runner.load(rlg_config_dict)
runner.reset()

self.seed = 42


In [4]:
agent = runner.create_player()

agent.restore(cfg.checkpoint)

agent.games_num = agent.env.num_environments

agent.run()

Obs type: full
[Warning] [carb.gym.plugin] useGpu is set, forcing single scene (0 subscenes)
Not connected to PVD
+++ Using GPU PhysX
Physics Engine: PhysX
Physics Device: cuda:0
GPU Pipeline: enabled
Num hand dofs:  17


/scr1/.pyenv/versions/miniconda3-latest/envs/rlgpu/lib/python3.7/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
  "Box bound precision lowered by casting to {}".format(self.dtype)


[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] Use

In [4]:
runner.run_play({"train": False,
                 "play": True,
                 "checkpoint": cfg.checkpoint,
                 "sigma": None})

Started to play
Obs type: full


/scr1/.pyenv/versions/miniconda3-latest/envs/rlgpu/lib/python3.7/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
  "Box bound precision lowered by casting to {}".format(self.dtype)


[Warning] [carb.gym.plugin] useGpu is set, forcing single scene (0 subscenes)
Not connected to PVD
+++ Using GPU PhysX
Physics Engine: PhysX
Physics Device: cuda:0
GPU Pipeline: enabled
Num hand dofs:  17
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceeded
[Warning] [carb.gym.plugin] User aggregate size exceed

KeyboardInterrupt: 

## Test run one step
```python
obs = agent.env_reset(agent.env)
batch_size = agent.get_batch_size(obs['obs'], 1)
actions = agent.get_action(agent.obs_to_torch(obs))
obses, r, done, info = agent.env_step(agent.env, actions)
```